In [659]:
import requests
import os
from utils import (
    DATA_FOLDER,
    download_file,
    ggunzip,
    ESTACIONES_XLSX,
    PREDICTIONS_FOLDER
)
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_error

pd.options.display.max_rows = None

# Download NOAA data

In [62]:
URL = "https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/archive/daily-summaries-latest.tar.gz"

TAR_GZ = "daily-summaries-latest.tar.gz"
NOAA_DATA_FOLDER = 'daily-summaries-latest'

In [63]:
download_file(URL, os.path.join(DATA_FOLDER, TAR_GZ))

In [64]:
ggunzip(os.path.join(DATA_FOLDER, TAR_GZ),
       os.path.join(DATA_FOLDER, NOAA_DATA_FOLDER))

# Dowload stations information

In [65]:
STATIONS_DESC_URL = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"
STATIONS_DESC = "ghcnd-stations.txt"
download_file(STATIONS_DESC_URL, os.path.join(DATA_FOLDER, STATIONS_DESC))

In [66]:
# we just need the location to see if it's close to target station

lines = []
with open(os.path.join(DATA_FOLDER, STATIONS_DESC)) as f:
    for line in f:
        lines.append(line.split()[0:3])

In [638]:
df_est_noaa_desc = pd.DataFrame(lines, columns=["code", "lat", "lon"])
df_est_noaa_desc.head()

,code,lat,lon
0,ACW00011604,17.1167,-61.7833
1,ACW00011647,17.1333,-61.7833
2,AE000041196,25.3330,55.5170
3,AEM00041194,25.2550,55.3640
4,AEM00041217,24.4330,54.6510


### distance to San Luis Tucuman


In [588]:
from geopy.distance import geodesic
def km_geodesic(x, y):
    return geodesic(x, y).km

from functools import partial

In [648]:
df_info_est = pd.read_excel(ESTACIONES_XLSX, sheet_name=0, header=0)
df_info_est = df_info_est[df_info_est.Estacion == 'San Luis Tucuman']
df_info_est[['lat', 'lon']] = -df_info_est['LAT (S), LONG (W)(º)'].str.split(',', expand=True).astype(float)
slt_loc = df_info_est[['lat', 'lon']].values[0]

In [649]:
dist_to_slt = partial(km_geodesic, slt_loc)

In [650]:
df_est_noaa_desc['dist_to_slt'] = df_est_noaa_desc[['lat', 'lon']].apply(lambda x: tuple(x), axis=1).apply(dist_to_slt)

In [651]:
df_est_noaa_desc = df_est_noaa_desc.sort_values(by=['dist_to_slt'])
df_est_noaa_desc.head(10)

,code,lat,lon,dist_to_slt,inv_dist_to_slt
288,ARM00087121,-26.8410,-65.1050,9.037911,0.110645
264,AR000087129,-27.7670,-64.3000,122.300301,0.008177
294,ARM00087222,-28.5960,-65.7520,204.430796,0.004892
280,AR000870470,-24.8500,-65.4830,228.548280,0.004375
287,ARM00087046,-24.3930,-65.0980,274.519972,0.003643
266,AR000087217,-29.3830,-66.8170,329.725849,0.003033
293,ARM00087213,-29.2330,-67.4330,353.325080,0.002830
295,ARM00087244,-29.9000,-63.6830,360.453629,0.002774
262,AR000087065,-24.1670,-62.9000,367.510826,0.002721
298,ARM00087320,-30.3670,-66.2830,406.765686,0.002458


In [652]:
df_est_noaa_desc = df_est_noaa_desc[df_est_noaa_desc.dist_to_slt <= 300.0]

In [654]:
df_est_noaa_desc['inv_dist_to_slt'] = 1. / df_est_noaa_desc.dist_to_slt

## read data from stations closer than 300km

In [655]:
station_codes = set(df_est_noaa_desc.code.values)

In [656]:
station_codes

{'AR000087129', 'AR000870470', 'ARM00087046', 'ARM00087121', 'ARM00087222'}

In [95]:
iter_csv = pd.read_csv(
    os.path.join(DATA_FOLDER, NOAA_DATA_FOLDER),
    skiprows=1, 
    names=["STATION","DATE","LATITUDE","LONGITUDE",
            "ELEVATION","NAME","PRCP","PRCP_ATTRIBUTES",
            "SNOW","SNOW_ATTRIBUTES","SNWD","SNWD_ATTRIBUTES",
            "TMAX","TMAX_ATTRIBUTES","TMIN","TMIN_ATTRIBUTES",
            "TOBS","TOBS_ATTRIBUTES"],
    iterator=True,
    chunksize=10000
)

In [99]:
from tqdm import tqdm_notebook
import tqdm

In [100]:
df = pd.concat([chunk[chunk.iloc[:,0].isin(station_codes)] for chunk in tqdm_notebook(iter_csv)])

/home/nox/repos/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [286]:
df.to_pickle(os.path.join(DATA_FOLDER, 'filtered_daily_summaries.pkl'))
df.PRCP = df.PRCP.astype(float)

# Closest station as proxy

In [674]:
closest = df[(df.STATION == "ARM00087121")&(~df.PRCP.isnull())][['STATION', 'DATE', 'PRCP']]
closest['year'] = closest.DATE.str.split('-').apply(lambda x: int(x[0]))
closest = closest.groupby(['year'])[['PRCP']].max() / 10.
df_maximos = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1)[['Año hid', 'San Luis Tucuman']]
df_cmp = pd.merge(df_maximos, closest, how='inner', left_on='Año hid', right_on='year')
df_cmp['ratio'] = df_cmp['PRCP'] / df_cmp['San Luis Tucuman']
df_cmp[df_cmp['Año hid'] > 1981]

,Año hid,San Luis Tucuman,PRCP,ratio
1,1982,62.0,161.0,2.596774
2,1983,122.0,120.9,0.990984
3,1984,96.0,56.9,0.592708
4,1985,94.0,0.0,0.000000
5,1986,50.0,16.0,0.320000
6,1987,51.0,150.1,2.943137
7,1988,90.0,101.1,1.123333
8,1989,179.0,64.0,0.357542
9,1990,45.0,104.9,2.331111
10,1991,67.0,181.1,2.702985


In [675]:
mean_absolute_error(df_cmp[-10:]['San Luis Tucuman'], df_cmp[-10:].PRCP)

109.1

# [Collection of Historical Weather Data: Issues with Missing Values](https://arxiv.org/abs/1910.08626)

## Normal ratio with geographical coordinates method 

$$Y_s = \sum_{i=1}^{N}\frac{ \frac{1}{x_i^2+y_i^2} \frac{M_s}{M_i} }{\sum_{i=1}^{N} \frac{1}{x_i^2+y_i^2} \frac{M_s}{M_i} }Y_i$$

In [615]:
df_daily_data = df.pivot_table(values='PRCP', columns='STATION', index='DATE', aggfunc='sum')
df_daily_data = df_daily_data[df_est_desc.code]
df_daily_data = df_daily_data.fillna(0.0)
df_daily_data.head()

STATION,AR000087129,AR000870470,ARM00087046,ARM00087121,ARM00087222
DATE,,,,,
1956-02-01,0.0,0.0,0.0,0.0,0.0
1956-02-02,0.0,0.0,0.0,0.0,0.0
1956-02-03,0.0,0.0,0.0,0.0,0.0
1956-02-04,0.0,0.0,0.0,0.0,0.0
1956-02-05,0.0,0.0,0.0,0.0,0.0


### $M_s$

In [616]:
df_anuales = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1)
df_anuales = df_anuales[df_anuales['Año hid'] >= 1973][['Año hid', 'San Luis Tucuman']].reset_index(drop=True)
df_anuales['daily_avg'] = df_anuales['San Luis Tucuman'].cumsum() / ((1 + df_anuales['San Luis Tucuman'].index) * 365)
M_s = df_anuales['San Luis Tucuman'].mean() / 365

### El resto

In [617]:
Y_cols = [f"{col}_Y" for col in df_daily_data.columns]
M_cols = [f"{col}_M" for col in df_daily_data.columns]
df_daily_data.rename(inplace=True, columns=lambda x: f"{x}_Y")
df_daily_data[M_cols] = df_daily_data[Y_cols]

In [618]:
# M_i calculation
df_daily_data[M_cols] = df_daily_data[M_cols].cumsum(axis=0).divide(np.arange(1, len(df_daily_data)+1), axis=0)

In [619]:
# M_s / M_i coefficient
df_daily_data[M_cols] = M_s / df_daily_data[M_cols]

In [620]:
# (M_s / M_i) * inverse distance
df_daily_data[M_cols] = df_daily_data[M_cols].multiply(df_est_desc.dist_to_slt.values, axis='columns')

In [621]:
# sum M values
df_daily_data['sum_Mi'] = df_daily_data[M_cols].sum(axis=1)

In [622]:
# filter relevant dates
df_daily_data = df_daily_data[(df_daily_data[Y_cols].fillna(0.) != 0.).cumsum(axis=0).all(axis=1)]

In [623]:
# normalize M_i values
df_daily_data[M_cols] = df_daily_data[M_cols].divide(df_daily_data.sum_Mi, axis=0)

In [624]:
df_daily_data['NRGC'] = (df_daily_data[Y_cols].values * df_daily_data[M_cols].values).sum(axis=1)

In [625]:
df_daily_data['year'] = df_daily_data.index.str.split('-').map(lambda x: x[0])

In [626]:
predictions = df_daily_data.groupby('year')[['NRGC']].max() / 10.
predictions.index = predictions.index.astype(int)

In [627]:
df_maximos = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1)[['Año hid', 'San Luis Tucuman']]

In [628]:
df_cmp = pd.merge(df_maximos, predictions, how='outer', left_on='Año hid', right_on='year')
df_cmp['ratio'] = df_cmp['NRGC'] / df_cmp['San Luis Tucuman']
df_cmp[df_cmp['Año hid'] > 1981]

,Año hid,San Luis Tucuman,NRGC,ratio
36,1982.0,62.0,159.939186,2.579664
37,1983.0,122.0,116.366490,0.953824
38,1984.0,96.0,54.797566,0.570808
39,1985.0,94.0,2.355075,0.025054
40,1986.0,50.0,15.675750,0.313515
41,1987.0,51.0,142.859045,2.801158
42,1988.0,90.0,94.846235,1.053847
43,1989.0,179.0,59.541163,0.332632
44,1990.0,45.0,98.916374,2.198142
45,1991.0,67.0,164.688295,2.458034


In [634]:
predictions[(predictions.index >= 2006)
            &(predictions.index <= 2015)
].reset_index(
    drop=True
).to_csv(
    os.path.join(PREDICTIONS_FOLDER, 'NOAA.csv'),
    header=False
)